In [1]:
from oo_cqed_rhf import CQEDRHFCalculator
import numpy as np
import psi4

In [2]:
lambda_vector = [0.0, 0.0, 0.05]
# molecule string for H2O
molecule_string = """

0 1
    O      0.000000000000   0.000000000000  -0.068516219320
    H      0.000000000000  -0.790689573744   0.543701060715
    H      0.000000000000   0.790689573744   0.543701060715
no_reorient
symmetry c1
"""

# options for H2O
psi4_options = {
    "basis": "cc-pVDZ",
    "save_jk": True,
    "scf_type": "pk",
    "e_convergence": 1e-12,
    "d_convergence": 1e-12,
}


In [3]:
calc = CQEDRHFCalculator(lambda_vector, molecule_string, psi4_options)
calc.calc_cqed_rhf_energy()
calc.summary()


Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Sun Jun 15 11:49:11 2025

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ----------------------------------------------

In [4]:
calc.summary()

RHF Energy:           -76.02141845 Ha
CQED-RHF Energy:      -76.01635528 Ha
Dipole Energy:        0.00004568 Ha
Nuclear Repulsion:    8.80146556 Ha
Dipole Moment:        [-1.00719147e-15 -4.44089210e-16  8.27920747e-01]


In [5]:
expected_h2o_e = -76.016355284146
assert np.isclose(expected_h2o_e, calc.cqed_rhf_energy)

In [6]:
calc.calc_scf_gradient(qed_wfn=True)


*** tstart() called on CHEM9QDFT72ALT
*** at Sun Jun 15 11:49:11 2025


         ------------------------------------------------------------
                                   SCF GRAD                          
                          Rob Parrish, Justin Turney,                
                       Andy Simmonett, and Alex Sokolov              
         ------------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   -----------------  -----------------  -----------------  -----------------
         O            0.000000000000     0.000000000000    -0.068516219320    15.994914619570
         H            0.000000000000    -0.790689573744     0.543701060715     1.007825032230
         H            0.000000000000     0.790689573744     0.54

array([[-1.83616648e-16,  2.04214294e-15, -6.01300796e-02],
       [ 2.98930327e-16, -4.33923429e-02,  3.00650398e-02],
       [-1.15313679e-16,  4.33923429e-02,  3.00650398e-02]])

r repulsion =    8.801465564563406

  ==> Basis Set <==

  Basis Set: CC-PVDZ
    Blend: CC-PVDZ
    Number of shells: 12
    Number of basis functions: 24
    Number of Cartesian functions: 25
    Spherical Harmonics?: true
    Max angular momentum: 2

  ==> DirectJKGrad: Integral-Direct SCF Gradients <==

    Gradient:                    1
    J tasked:                  Yes
    K tasked:                  Yes
    wK tasked:                  No
    Integrals threads:           1
    Schwarz Cutoff:          1E-12


  -Total Gradient:
     Atom            X                  Y                   Z
    ------   -----------------  -----------------  -----------------
       1       -0.000000000000     0.000000000000    -0.060130079561
       2        0.000000000000    -0.043392342878     0.030065039780
       3       -0.000000000000     0.043392342878     0.030065039780


*** tstop() called on CHEM9QDFT72ALT at Sun Jun 15 11:49:11 2025
Module time:
	user time   =       0.01 seconds =       

In [7]:
mol = psi4.geometry(calc.molecule_string)
psi4.set_options(calc.psi4_options)
gradient = np.asarray(psi4.gradient("scf"))



Scratch directory: /tmp/
gradient() will perform analytic gradient computation.
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Sun Jun 15 11:49:11 2025

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core


In [8]:
print(calc.scf_grad -gradient)

[[-5.99080176e-19  3.55801311e-15 -5.44902036e-04]
 [ 2.79143728e-16 -3.45885116e-04  2.72451018e-04]
 [-2.78544648e-16  3.45885116e-04  2.72451018e-04]]


In [9]:
calc.calc_quadrupole_gradient()

[[ 1.32601432e-18  9.99294165e-19 -2.77573793e-03]
 [ 1.03219418e-19  2.17498212e-04  1.62681438e-03]
 [-1.42923374e-18 -2.17498212e-04  1.62681438e-03]]
